In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np
import plotly.graph_objects as go

In [2]:
# Use for animation rotation at the end
x_eye = -1.25
y_eye = 2
z_eye = 0.5

In [3]:
df = pd.read_excel('./source/ws_mother_file.xlsx')
df

# df = df[df['Year'].isin(['2010'])]
# df = df[df['Continent'].isin(['Africa', 'Europe'])]
# # df = df[df['Year'].isin(['1990','1995','2000','2005','2010'])]
# df['resized_pop'] = df['population'] / 100000000  # use for size parameter

,WORK_STAND_NM,WORK_UNIT,WORK_ACT,CHK_INFO,부서,대구분,중구분,소구분
0,선각 철의장풍/치공구류 제작,작업 준비,당일 제작 수량 확인,관련 도면준비,가공소조립부,가공,가공/제작,기계가공
1,선각 철의장풍/치공구류 제작,작업 준비,"필요 자재(잔재, 원자재) 파악",자재 부족시 관련 부서 협조 요청,가공소조립부,가공,가공/제작,기계가공
2,선각 철의장풍/치공구류 제작,작업 준비,장비별 작업자 배원,NaN,가공소조립부,가공,가공/제작,기계가공
3,선각 철의장풍/치공구류 제작,물량 배정,수동 마킹(NC 마킹의 반대 개념),NaN,가공소조립부,가공,가공/제작,기계가공
4,선각 철의장풍/치공구류 제작,물량 배정,수동 절단(기계 절단의 반대 개념),NaN,가공소조립부,가공,가공/제작,기계가공
...,...,...,...,...,...,...,...,...
23528,사내/생산현장 이동,자전거 운전,블록하부 통행 및 크레인 주행구간 등 작업반경 내 주행 금지,NaN,조선해양품질경영2부,기능,공통,이동
23529,사내/생산현장 이동,자전거 운전,우천시 우의 착용 후 운행,우산사용 금지,조선해양품질경영2부,기능,공통,이동
23530,사내/생산현장 이동,자전거 운전,크레인 레일 위 이동시에는 레일 방향의 직각으로 이동,레일 사이에 바퀴가 빠져 넘어질 수 있음,조선해양품질경영2부,기능,공통,이동
23531,사내/생산현장 이동,자전거 운전,목적지 도착 후 지정된 주차장소에 주차,NaN,조선해양품질경영2부,기능,공통,이동


In [4]:
# dept = ["가공소조립부", "판넬조립부", "대조립1부", "대조립2부", "의장생산부", "선행의장부", "선행도장부",
#        "선실생산부", "건조1부", "건조2부", "건조3부", "의장1부", "의장2부", "의장3부", "CHS공사부", "도장1부", "도장2부",
#        "LNG공사부", "시운전부", "운항관제과", "외업공사지원부", "발판지원부", "자재운영부", "조선해양품질경영1부", "조선해양품질경영2부"]
# key =[n for n in range(len(dept))]
# value = [v for v in dept]

In [5]:
# order = pd.DataFrame(dept, columns=["부서"])
# order

In [6]:
dff = df.groupby(['대구분', '부서', 'WORK_STAND_NM'], as_index=False)['WORK_ACT'].count()
dff

,대구분,부서,WORK_STAND_NM,WORK_ACT
0,LNG공사,LNG공사부,CARGO TANK MEMB. MARKING,14
1,LNG공사,LNG공사부,CARGO TANK MEMB. PANEL보온 I/P 설치,14
2,LNG공사,LNG공사부,CARGO TANK MEMB. PANEL보온 TPX/TBP 설치,27
3,LNG공사,LNG공사부,CARGO TANK MEMB. S/OPENING 보온,9
4,LNG공사,LNG공사부,CARGO TANK MEMB. 설치 MEMB. FIT-UP,6
...,...,...,...,...
1399,후행의장,의장3부,철의장 검사_FAN & DAMPER OPERATION/VOLUME TEST,16
1400,후행의장,의장3부,철의장 검사_LIFE & RESCUE BOAT/DAVIT TEST,24
1401,후행의장,의장3부,함석시공,20
1402,후행의장,의장3부,후PE 전장 설치,44


In [7]:
dff2 = dff.groupby(['대구분', '부서'], as_index=False)['WORK_STAND_NM'].count()
dff2

,대구분,부서,WORK_STAND_NM
0,LNG공사,LNG공사부,17
1,QM,조선해양품질경영1부,16
2,QM,조선해양품질경영2부,14
3,가공,가공소조립부,7
4,건조,건조1부,12
...,...,...,...
86,후행의장,선실생산부,1
87,후행의장,의장1부,50
88,후행의장,의장2부,50
89,후행의장,의장3부,50


In [8]:
# dff2.to_excel('./source/groupby_data2.xlsx')

In [9]:
dff2 = pd.read_excel('./source/groupby_data2.xlsx')
dff2["size"] = dff2['WORK_STAND_NM']*1
dff2
# dff2 = dff2[dff2["대구분"] != "기능"]

,Unnamed: 0,대구분,부서,WORK_STAND_NM,size
0,3,가공,가공소조립부,7,7
1,75,소조립,가공소조립부,12,12
2,66,선행의장,가공소조립부,4,4
3,48,선실선행의장,가공소조립부,1,1
4,13,기능,가공소조립부,20,20
...,...,...,...,...,...
86,33,기능,자재운영부,8,8
87,1,QM,조선해양품질경영1부,16,16
88,34,기능,조선해양품질경영1부,3,3
89,2,QM,조선해양품질경영2부,14,14


In [49]:
fig = px.scatter_3d(
    data_frame=dff2,
    x='부서',
    y='대구분',
    z='WORK_STAND_NM',
    color="부서",
#     color_discrete_sequence=['magenta', 'green'],
    # color_discrete_map={'Europe': 'black', 'Africa': 'yellow'},
    opacity=0.5,              # opacity values range from 0 to 1
#     symbol='대구분',            # symbol used for bubble
    # symbol_map={"2005": "square-open", "2010": 3},
    size='size',       # size of bubble
    size_max=50,              # set the maximum mark size when using size
    log_z=True,  # you can also set log_y and log_z as a log scale
#     range_z=[9,13],           # you can also set range of range_y and range_x
    template='plotly_dark',         # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
    title='부서/공정별 작업표준 분포(로그 스케일)',
    labels={'WORK_STAND_NM': '작업표준 개수(로그)'},
    hover_data={'부서': False},
    # hover_data={'Continent': False, 'GDP per capita': ':.1f'},
    hover_name='부서',        # values appear in bold in the hover tooltip
    height=900,                 # height of graph in pixels
#     width=100,
    # animation_frame='Year',   # assign marks to animation frames
#     range_x=[-5,30],
#     range_z=[0,250],
#     range_y=[0,30]

)

# fig.update_traces(showlegend=False, selector=dict(type='scatter3d'))
# fig.update_traces(legendgroup="", selector=dict(type='scatter3d'))
fig.update_traces(marker_symbol="circle", selector=dict(type='scatter3d'))
##( "circle" | "circle-open" | "square" | "square-open" | "diamond" | "diamond-open" | "cross" | "x" )

pio.show(fig)
title="부서_공정별 작업표준 분포(로그 스케일)"
fig.write_html("./results/{}.html".format(title))


In [11]:
# fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
# fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 500

In [12]:
# Use for animation rotation
# fig.update_layout(scene_camera_eye=dict(x=x_eye, y=y_eye, z=z_eye),
#                   updatemenus=[dict(type='buttons',
#                                     showactive=False,
#                                     y=1,
#                                     x=0.8,
#                                     xanchor='left',
#                                     yanchor='bottom',
#                                     pad=dict(t=45, r=10),
#                                     buttons=[dict(label='Play',
#                                                   method='animate',
#                                                   args=[None, dict(frame=dict(duration=250, redraw=True),
#                                                                    transition=dict(duration=0),
#                                                                    fromcurrent=True,
#                                                                    mode='immediate'
#                                                                    )]
#                                                   )
#                                              ]
#                                     )
#                                ]
#                   )
#
#


In [13]:
# def rotate_z(x, y, z, theta):
#     w = x + 1j * y
#     return np.real(np.exp(1j * theta) * w), np.imag(np.exp(1j * theta) * w), z

# frames = []

# for t in np.arange(0, 6.26, 0.1):
#     xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
#     frames.append(go.Frame(layout=dict(scene_camera_eye=dict(x=xe, y=ye, z=ze))))
# fig.frames = frames


# # fig.write_html("My3dPlot.html")
# pio.show(fig)

# Graph Object 방식

In [14]:
trace1 = go.Scatter3d(
    x=dff2['부서'],
    y=dff2['대구분'],
    z=dff2['WORK_STAND_NM'],
    mode='markers+text',
    text=dff2['WORK_STAND_NM'],
    marker = dict(
    
    size=dff2['WORK_STAND_NM'], 
    color=dff2["WORK_STAND_NM"],
    colorscale='Viridis',
    opacity=0.5,
    showscale=True))

data = [trace1]

layout = go.Layout(
    margin = dict(l=0, r=0, b=0, t=0),
    )

In [15]:
fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_html("./results/test9_3d.html")